# Segmentation panoptique des noyaux et des tissus dans le mélanome avancé (PUMA)
## Projet deep learning

Bonjour à tous,

Nous allons vous présenter notre projet de deep learning sur la segmentation et classification des noyaux cellulaires dans le mélanome.

Dans ce projet, nous avons travaillé sur l'application de méthodes et modèles de deep  sur la segmentation et classification des noyaux cellulaires et tissus dans les images histopathologiques du mélanome en utilisant des techniques de deep learning.

Ce projet se base sur le concours PUMA organisé par le consortium de recherche sur le mélan

Nous nous basons sur l'article scientifique : "PUMA: A dataset for deep learning-based nuclei and tissue segmentation in advanced melanoma with improved biomarker potential" publié dans GigaScience en 2024.

Ce travail s'inscrit dans une démarche d'amélioration des biomarqueurs pronostiques pour le mélanome avancé.

# 1. Contexte

## Le mélanome et ses biomarqueurs

- Mélanome : cancer agressif de la peau en augmentation
- 50% des patients ne répondent pas à l'immunothérapie
- Les lymphocytes infiltrant la tumeur (TILs) = biomarqueur pronostique clé
- Limites actuelles : évaluation manuelle subjective et variable

Le mélanome est une forme particulièrement agressive de cancer cutané dont l'incidence augmente globalement. Bien que le mélanome primaire soit souvent traité par excision chirurgicale, le mélanome avancé nécessite une immunothérapie par inhibiteurs de points de contrôle immunitaires.

Malheureusement, environ la moitié des patients ne répondent pas à cette thérapie, qui est coûteuse et potentiellement toxique. Il est donc crucial d'identifier des biomarqueurs capables de prédire la réponse au traitement.

Les lymphocytes infiltrant la tumeur (TILs) sont reconnus comme un biomarqueur pronostique important, mais leur évaluation manuelle est subjective, chronophage et souffre d'une variabilité inter-observateurs considérable.

Ces limitations motivent le développement d'approches automatisées basées sur l'intelligence artificielle pour quantifier et caractériser les TILs de manière plus objective et reproductible.

## Défis Spécifiques au Mélanome

- Cellules de mélanome mimétiques imitant d'autres types cellulaires
- Modèles génériques (HoverNet sur PanNuke) : performance sous-optimale
- Confusion fréquente entre cellules tumorales/lymphocytes/stromales

→ Nécessité d'un modèle spécifique au mélanome

L'analyse histopathologique du mélanome présente des défis spécifiques qui compliquent l'application des modèles d'IA existants :

1. Les cellules de mélanome ont une capacité unique à imiter morphologiquement d'autres types cellulaires, rendant leur identification automatique particulièrement difficile.

2. Les modèles génériques comme HoverNet, même lorsqu'ils sont pré-entraînés sur des datasets comme PanNuke (qui inclut des échantillons de peau), montrent des performances sous-optimales sur les images de mélanome.

3. Ces modèles confondent fréquemment les cellules tumorales avec des lymphocytes ou des cellules stromales, compromettant ainsi l'évaluation précise des TILs.

4. Les méthodes manuelles actuelles souffrent également d'une grande variabilité dans l'évaluation de la localisation des TILs (intratumorale vs péritumorale), qui est pourtant cruciale pour évaluer leur valeur pronostique.

Ces défis justifient le développement d'un modèle spécifiquement adapté au mélanome, entraîné sur des données annotées de haute qualité.

## Jeu de données

Le jeu de données PUMA : 

- 310 échantillons:
  - 155 ROIs de mélanomes primaires + 155 ROIs métastatiques
  - Images H&E à 40x (1024×1024 pixels)
  - Annotations validées par un dermato-pathologiste
    - Noyaux : tumeur, lymphocytes, plasmocytes, histiocytes (10 classes)
    - Tissus : tumeur, stroma, épiderme, vaisseaux sanguins (5 classes)

Le dataset PUMA (Panoptic segmentation of nUclei and tissue in MelanomA) constitue la première base de données spécifique au mélanome pour la segmentation panoptique des noyaux cellulaires et des tissus :

- Il comprend 155 régions d'intérêt (ROIs) de mélanomes primaires et 155 de mélanomes métastatiques, offrant ainsi un échantillonnage représentatif de la diversité morphologique de cette pathologie.

- Les images sont des coupes histologiques colorées à l'hématoxyline et à l'éosine (H&E), numérisées à un grossissement de 40x avec une résolution de 1024×1024 pixels.

- Chaque ROI est accompagnée d'une ROI contextuelle de 5120×5120 pixels centrée autour d'elle, permettant d'analyser l'environnement tissulaire plus large.

- Les annotations des noyaux cellulaires identifient 10 types distincts : tumeur, lymphocytes, plasmocytes, histiocytes, mélanophages, neutrophiles, cellules stromales, épithélium, endothélium et cellules apoptotiques.

- Les annotations tissulaires comprennent 5 catégories : tumeur, stroma, épithélium, vaisseaux sanguins et zones nécrotiques.

- Toutes les annotations ont été réalisées par un professionnel médical et vérifiées par un dermato-pathologiste certifié, garantissant leur qualité et leur pertinence clinique.

Ce jeu de données unique permet de développer et d'évaluer des modèles spécifiques au mélanome avec un niveau de granularité sans précédent.

## Principe

Il y a 2 tâches : 

1. Segmentation et classification des noyaux cellulaires
2. Segmentation et classification des tissus

Ces deux tâches sont différentes mais complémentaires.

Pour permettre le diagnostic et le pronostic du mélanome, il est essentiel de segmenter et classifier les noyaux cellulaires et les tissus dans les images histopathologiques.

Il s'agit de deux tâches de segmentation panoptique, où chaque pixel d'une image est classé en fonction de son appartenance à une catégorie spécifique.

Attention, il faut bien faire la différence entre la segmentation panoptique et la segmentation sémantique. 

La segmentation panoptique consiste à segmenter chaque pixel d'une image en lui attribuant une étiquette de classe, tout en tenant compte de la hiérarchie des objets présents dans l'image. En revanche, la segmentation sémantique se concentre uniquement sur l'attribution d'étiquettes de classe aux pixels, sans tenir compte de la hiérarchie ou des relations entre les objets.

Ainsi, il à la fois une tâche de segmentation sémantique (identifier les classes) et une tâche de détection d'objets (identifier les instances).

Il faut donc 2 modèles : un pour la segmentation des noyaux et un pour la segmentation des tissus.

# 2. Méthodologie

1. Etat de l'art 
2. Prétraitement
3. Modèle
4. Entraînement
5. Évaluation
6. Interprétation

La méthodologie de ce projet se décline en plusieurs étapes clés :

1. **État de l'art** : Analyse des méthodes existantes pour la segmentation et la classification des noyaux cellulaires et des tissus dans les images histopathologiques, en mettant l'accent sur les défis spécifiques au mélanome. Cette étape est réalisée en consultant la littérature scientifique et également en demandant à ChatGPT de faire une revue de la littérature sur le sujet.
2. **Prétraitement** : Préparation des données, y compris la normalisation des images, l'augmentation des données et la création de masques d'annotation pour les noyaux et les tissus.
3. **Modèle** : Sélection et adaptation d'un modèle de deep learning pour la segmentation panoptique, en tenant compte des spécificités du mélanome.
4. **Entraînement** : Entraînement du modèle sur le jeu de données PUMA, en utilisant des techniques d'optimisation et de régularisation pour améliorer la performance.
5. **Évaluation** : Évaluation des performances du modèle sur un ensemble de test indépendant, en utilisant des métriques appropriées pour la segmentation et la classification.
6. **Interprétation** : Analyse des résultats obtenus, identification des forces et des faiblesses du modèle, et discussion des implications cliniques potentielles.

# 3. Projet

Nous avons essayé plusieurs 

# 4. Résultats

# 5. Conclusion

## Critique

- **Qualité de l'implémentation :**
  - Gestion insuffisante des erreurs.
  - Manque de clarté et de robustesse du code.
- **Stabilité et Dépendances :**
  - Difficultés liées à la gestion des versions des librairies.
  - Instabilité potentielle des environnements d'exécution.
- **Maintenance et Support :**
  - Peu de suivi des problèmes signalés (issues).
  - Manque de réactivité sur les contributions (pull requests).

Au cours de ce projet, l'exploitation des modèles existants a présenté plusieurs défis.

Premièrement, la qualité de l'implémentation de certains outils s'est avérée perfectible. Par exemple, une gestion limitée des exceptions a entraîné des arrêts inattendus du programme, parfois sans message d'erreur explicite, compliquant le débogage.

Deuxièmement, la stabilité des modèles a été affectée par une gestion imprécise des dépendances. Il a fallu tester itérativement différentes versions de librairies pour assurer la compatibilité et le bon fonctionnement, ce qui a ralenti le processus de développement.

Enfin, le manque de maintenance active et de support communautaire pour certains des outils utilisés a été un frein. Les problèmes signalés par la communauté (issues GitHub) restent souvent sans réponse, et les propositions d'amélioration (pull requests) ne sont pas intégrées, ce qui soulève des questions sur la pérennité et la fiabilité à long terme de ces outils.

## Impact

- **Impact calcul et environnemental :**
  - **Intensité calculatoire :** Utilisation massive d'opérations matricielles (propagation avant, calcul de perte, rétropropagation, mise à jour des poids) nécessitant des accélérateurs matériels (GPU/TPU).
  - **Utilisation des ressources GPU :** Forte demande en mémoire VRAM pour stocker les poids du modèle et les données d'entraînement/inférence.
  - **Consommation énergétique :** L'entraînement et l'inférence des modèles profonds sont énergivores. Des architectures comme CellViT++, basées sur des modèles pré-entraînés, peuvent être plus économes que des entraînements complets comme avec nnU-Net.

- **Impact politique :** Nvidia et AMD font fabriquer leurs GPU à Taïwan, ce qui soulève des questions sur la dépendance technologique et les implications géopolitiques. 

- **Impact Clinique et Sociétal :**
  - Amélioration du diagnostic/pronostic
  - Optimisation du flux de travail
  - Aide à la décision thérapeutique
  - Potentiel de réduction des coûts

L'impact de ce type de projet se manifeste à plusieurs niveaux.

D'un point de vue **calcul et environnemental**, les modèles de deep learning reposent sur des calculs matriciels intensifs. L'entraînement et même l'inférence exigent des ressources matérielles spécialisées, notamment des GPU dotés d'une mémoire VRAM importante et d'une puissance de calcul élevée. Ces exigences entraînent une consommation énergétique non négligeable, bien que variable selon l'architecture du modèle et la phase (entraînement vs inférence). L'impact environnemental dépend fortement de la source d'énergie utilisée pour alimenter les centres de calcul. Des optimisations (modèles plus légers, inférence sur du matériel dédié moins énergivore) sont des pistes pour mitiger cet impact.

D'un point de vue **politique**, la dépendance technologique vis-à-vis de fabricants de GPU comme Nvidia et AMD, qui externalisent leur production à Taïwan, soulève des questions sur la souveraineté numérique et les implications géopolitiques. La concentration de la fabrication de composants critiques dans certaines régions du monde peut poser des risques en cas de tensions internationales. Cependant, les machines utilisées pour la fabrication de ces GPU sont souvent conçues et assemblées en Europe (ASML), ce qui pourrait offrir une certaine résilience à cette dépendance technologique.

Sur le plan **clinique et sociétal**, l'automatisation de l'analyse d'images histopathologiques via le deep learning offre des perspectives prometteuses. Elle permet une quantification plus rapide, objective et reproductible de caractéristiques cellulaires et tissulaires complexes, difficiles à évaluer manuellement de manière standardisée. Cela peut conduire à une meilleure stratification des patients, à une aide à la décision thérapeutique plus éclairée (par exemple, pour prédire la réponse à l'immunothérapie dans le cas du mélanome) et à une optimisation du temps précieux des pathologistes. À terme, ces avancées pourraient améliorer la prise en charge des patients, potentiellement réduire les coûts de santé et contribuer à une médecine plus personnalisée.